# Using FalkorGraphRagCapability with UserProxyAgent for Graph RAG Question Answering

AutoGen provides graph rag integration with Agent Capability. This is an example to integrate FalkorDB (a Knowledge Graph Database).

## Install Falkor DB SDK

In [17]:
%pip install graphrag_sdk==0.1.3b0

# For debug only,
# %pip install -e /workspaces/autogen

1222214.22s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


1222219.78s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///workspaces/autogen
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for autogen (pyproject.toml) ... done
  Created wheel for autogen: filename=autogen-0.3.2-0.editable-py3-none-any.whl size=16216 sha256=6a1d2928c2581b5d66f17c8cafed5444dab21fcb5a52b4ba34fe43abbc4055ec
  Stored in directory: /tmp/pip-ephem-wheel-cache-9wh6yuuq/wheels/d4/86/1f/a538740a449c67aa3ff8343698c29d70fc94236f70bde92144
Successfully built autogen
  Attempting uninstall: autogen
    Found existing installation: autogen 0.3.0
    Uninstalling autogen-0.3.0:
      Successfully uninstalled autogen-0.3.0

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the ke

## Set OpenAI API

In [8]:
import os

import autogen

config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST")
os.environ["OPENAI_API_KEY"] = config_list[0]["api_key"]

## Create Knowledge Graph with Your Own Data

Note that, you would need to have a Falkor DB running ready. 
In this example, Falker DB endpint is already set at host="172.18.0.3" and port=6379.
For how to set up Falkor DB, please refer to https://docs.falkordb.com/

In [10]:
from graphrag_sdk.schema import Schema

from autogen.agentchat.contrib.graph_rag.document import Document, DocumentType
from autogen.agentchat.contrib.graph_rag.falkor_graph_query_engine import FalkorGraphQueryEngine

# Auto generate graph schema from unstructured data
input_path = "../test/agentchat/contrib/graph_rag/the_matrix.txt"

movie_schema = Schema()
actor = movie_schema.add_entity("Actor").add_attribute("name", str, unique=True)
movie = movie_schema.add_entity("Movie").add_attribute("title", str, unique=True)
movie_schema.add_relation("ACTED", actor, movie)

query_engine = FalkorGraphQueryEngine(
    name="IMDB",
    host="172.18.0.3",
    port=6379,
    schema=movie_schema,
)

input_documents = [Document(doctype=DocumentType.TEXT, path_or_url=input_path)]

query_engine.init_db(input_doc=input_documents)

## Create a UserProxyAgent for FalkorDB and Answer Questions

In [18]:
from autogen import UserProxyAgent
from autogen.agentchat.contrib.graph_rag.falkor_graph_rag_capability import FalkorGraphRagCapability

graph_rag_agent = UserProxyAgent(
    name="user_proxy",
    code_execution_config=False,
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
    human_input_mode="ALWAYS",
)
graph_rag_capability = FalkorGraphRagCapability(query_engine)
graph_rag_capability.add_to_agent(graph_rag_agent)

graph_rag_agent.initiate_chat(graph_rag_agent, message="Name a few actors who've played in 'The Matrix'")

user_proxy (to user_proxy):



Name a few actors who've played in 'The Matrix'

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
user_proxy (to user_proxy):



--------------------------------------------------------------------------------
user_proxy (to user_proxy):

List additional actors

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': "Name a few actors who've played in 'The Matrix'", 'role': 'assistant', 'name': 'user_proxy'}, {'content': "A few actors who have played in 'The Matrix' are:\n\n- Keanu Reeves\n- Laurence Fishburne\n- Carrie-Anne Moss\n- Hugo Weaving", 'role': 'user', 'name': 'user_proxy'}, {'content': '', 'role': 'assistant', 'name': 'user_proxy'}, {'content': None, 'role': 'user', 'name': 'user_proxy'}, {'content': 'List additional actors', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'The additional actors in the knowledge graph, ordered alphabetically by name, are:\n\n- Carrie-Anne Moss\n- Hugo Weaving\n- Keanu Reeves\n- Lana Wachowski\n- Laurence Fishburne\n- Lilly Wachowski', 'role': 'user', 'name': 'user_proxy'}], summary='The additional actors in the knowledge graph, ordered alphabetically by name, are:\n\n- Carrie-Anne Moss\n- Hugo Weaving\n- Keanu Reeves\n- Lana Wachowski\n- Laurence Fishburne\n- Lilly Wachowski', cost={'usage_incl